In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Set image dimensions and other parameters
image_size = (224, 224)  # MobileNetV2 expects 224x224 images
batch_size = 32
epochs = 50

# Define the paths to the dataset (adjust these paths as needed)
base_dir = './data'  # Root directory where the 'Angry', 'Happy', 'Sad', 'Fear' folders are located

# 1. **Data Augmentation**: Apply common transformations to prevent overfitting
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.5, 1.5],  # Random brightness adjustment
    validation_split=0.2  # This will split off 20% for validation
)

# Validation and Test generators (only rescaling)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# 2. **Load the data into train and validation sets using validation_split**
train_data_gen = train_datagen.flow_from_directory(
    base_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Multi-class classification (4 classes)
    shuffle=True,
    subset='training'  # This is the training data subset (80%)
)

val_data_gen = train_datagen.flow_from_directory(
    base_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Multi-class classification
    subset='validation'  # This is the validation data subset (20%)
)

# 3. **Use MobileNetV2 as a lighter model for smaller datasets**
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers initially
for layer in base_model.layers:
    layer.trainable = False

# Create the model with additional custom layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # L2 regularization
    layers.Dropout(0.5),  # Dropout to prevent overfitting
    layers.Dense(4, activation='softmax')  # Output layer with 4 categories
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 4. **Optionally, unfreeze the last few layers for fine-tuning**
for layer in base_model.layers[-20:]:  # Unfreeze last 20 layers, you can adjust this number
    layer.trainable = True

# Re-compile the model after unfreezing layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Reduced learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 5. **Setup callbacks for early stopping and model checkpointing**
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# 6. **Reduce learning rate if the validation loss does not improve**
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)



Found 322 images belonging to 4 classes.
Found 79 images belonging to 4 classes.


In [4]:
# 7. **Train the model**
history = model.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen,
    callbacks=[early_stop, checkpoint, lr_scheduler],
    verbose=1
)


Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.3166 - loss: 12.9564 - val_accuracy: 0.4684 - val_loss: 12.5706 - learning_rate: 1.0000e-04
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5083 - loss: 12.3498 - val_accuracy: 0.5190 - val_loss: 12.2516 - learning_rate: 1.0000e-04
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5739 - loss: 12.0590 - val_accuracy: 0.4304 - val_loss: 12.2156 - learning_rate: 1.0000e-04
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6174 - loss: 11.7995 - val_accuracy: 0.4937 - val_loss: 12.1216 - learning_rate: 1.0000e-04
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.7574 - loss: 11.4409 - val_accuracy: 0.4557 - val_loss: 12.1022 - learning_rate: 1.0000e-04
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.7067 - loss: 11.3275 - val_accuracy: 0.4684 - val_loss: 11.8992 - learning_rate: 1.0000e-04
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.

In [5]:

test_loss, test_acc = model.evaluate(val_data_gen)
print(f"Test accuracy: {test_acc:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 907ms/step - accuracy: 0.4607 - loss: 7.9667
Test accuracy: 0.4684
